In [5]:
import os
import joblib
import numpy as np
import cv2 # Usaremos OpenCV para cargar y preprocesar la imagen

def preprocess_single_image(image_path, target_size=(64, 64)):
    """
    Carga una imagen, la redimensiona a target_size y la convierte a escala de grises,
    y la aplana. DEBE ser consistente con el entrenamiento.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"La imagen no se encuentra en la ruta: {image_path}")

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) # Cargar en escala de grises

    if img is None:
        raise ValueError(f"No se pudo cargar la imagen. Comprueba la ruta y el formato: {image_path}")

    img = cv2.resize(img, target_size)

    img_flat = img.flatten()

    # Expandir las dimensiones para que sea (1, num_features) para el scaler y PCA
    return img_flat.reshape(1, -1)

# Directorio donde se guardaron los modelos
model_dir = 'saved_models'

# Cargar el scaler, PCA, el modelo KNN, los nombres de las etiquetas y el tamaño objetivo de la imagen
try:
    scaler = joblib.load(os.path.join(model_dir, 'scaler.pkl'))
    pca = joblib.load(os.path.join(model_dir, 'pca.pkl'))
    knn = joblib.load(os.path.join(model_dir, 'knn_model.pkl'))
    label_names = joblib.load(os.path.join(model_dir, 'label_names.pkl'))
    image_target_size = joblib.load(os.path.join(model_dir, 'image_target_size.pkl')) # Cargar el tamaño
    print("Modelos y parámetros cargados exitosamente.")
    print(f"El modelo espera imágenes de tamaño: {image_target_size}")
except FileNotFoundError as e:
    print(f"Error: No se encontraron los archivos del modelo en '{model_dir}'.")
    print(f"Detalle del error: {e}")
    print("Asegúrate de haber ejecutado 'train_and_save_model.py' primero.")
    exit()
except Exception as e:
    print(f"Error al cargar los modelos: {e}")
    exit()

# Ruta de la nueva imagen para clasificar
# ¡¡¡IMPORTANTE!!! CAMBIA ESTO A LA RUTA REAL DE TU IMAGEN
new_image_path = 'dogplaying.jpg' 

print(f"\nIntentando clasificar la imagen: {new_image_path}")

try:
    # Preprocesar la nueva imagen usando el target_size guardado
    processed_image = preprocess_single_image(new_image_path, target_size=image_target_size)

    # Normalizar la imagen usando el scaler entrenado
    scaled_image = scaler.transform(processed_image)

    # Aplicar PCA a la imagen normalizada
    pca_image = pca.transform(scaled_image)

    # Predecir la clase usando el modelo KNN
    prediction_index = knn.predict(pca_image)[0]
    predicted_label = label_names[prediction_index]
    
    # Opcional: Obtener probabilidades si el modelo las soporta (KNN no lo hace directamente para todas)
    # Si quisieras ver la distancia a los vecinos, tendrías que implementarlo manualmente con kneighbors

    print(f"\nLa imagen '{os.path.basename(new_image_path)}' se clasifica como: {predicted_label}")

except FileNotFoundError as e:
    print(e)
except ValueError as e:
    print(e)
except Exception as e:
    print(f"Ocurrió un error durante la clasificación: {e}")

Modelos y parámetros cargados exitosamente.
El modelo espera imágenes de tamaño: (64, 64)

Intentando clasificar la imagen: dogplaying.jpg

La imagen 'dogplaying.jpg' se clasifica como: entretenimiento
